In [56]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F


In [57]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
GeForce RTX 2080 with Max-Q Design


In [58]:
# data_dir = '../../../../BottleStoodUp_atNight/Positive/'        #For the work laptop
data_dir = '../../../Images/BottleStoodUp_atNight/Positive'        #For the home laptop

In [59]:
# transform_characteristics = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor()])

transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset = datasets.ImageFolder(data_dir, transform=transform_characteristics)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Building the network in which the pre-trained model will be loaded.

In [60]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # 32, 1, 224, 224.  Batch size, input channels, shape of the image.
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image divided by 2 approximately.   
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> 15, 110, 44, 44. Batch size, channel output, output image shape.
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # Dimension rseult: -> 15, 450, 1, 1
        )
        
        # Initial dimension for this part of the model: 15 , 450, 1, 1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(output_channels_layer3, output_channels_layer2, 8),  # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels_layer2, output_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # Dimension rseult: -> 15, 110, 44, 44
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels, input_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # Dimension rseult: -> 15, 1, 224, 224
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

In [61]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

device = get_device()
print(device)

cuda:0


In [62]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [63]:
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

Loading the model

In [64]:
filepath = "../../../BottlesAnomalies_TFM/models/pytorchModels/PytorchModel_withCUDA"
# For loading the model 
model.load_state_dict(torch.load(filepath))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

Extracting the layers' weights of the model that has been loaded.

In [65]:
layers_weights_list = []
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        weights = m.weight
        layers_weights_list.append(weights)
print("The number of layers that the loaded model has, is: ", len(layers_weights_list))

The number of layers that the loaded model has, is:  3


Copying the model layers' weights from one model to another. 

In [66]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # 32, 1, 224, 224.  Batch size, input channels, shape of the image.
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image divided by 2 approximately. 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> 15, 110, 44, 44. Batch size, channel output, output image shape.
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # Dimension rseult: -> 15, 450, 1, 1
        )

    def forward(self, x):
        # print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the show modules function")
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
            
    def show_one_layer_weights(self, index):
        print("This is the one layer show function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == index:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                i = i +1
    
    def update_weights(self):
        print("updating weights function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = layers_weights_list[i]
                i = i +1

                                        
 

In [67]:
model_encoder = Autoencoder_latentSpace()
model_encoder.to(device)
# model.eval()

Autoencoder_latentSpace(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
)

In [70]:
# Copy the layer weights from the base model to the target model
for i, (target_layer, base_layer) in enumerate(zip(model_encoder.children(), model.children())):
    target_params = target_layer.parameters()
    base_params = base_layer.parameters()

    # Copy the weights from the base model to the target model
    for target_param, base_param in zip(target_params, base_params):
        target_param.data.copy_(base_param.data)

# Verify if the weights are successfully copied
for target_layer, base_layer in zip(model_encoder.children(), model.children()):
    target_params = target_layer.parameters()
    base_params = base_layer.parameters()

    for target_param, base_param in zip(target_params, base_params):
        print(target_param.data.names)
        if torch.allclose(target_param.data, base_param.data):
            print("Layer weights are successfully copied.")
        else:
            print("Error: Layer weights are not copied correctly.")


(None, None, None, None)
Layer weights are successfully copied.
(None,)
Layer weights are successfully copied.
(None, None, None, None)
Layer weights are successfully copied.
(None,)
Layer weights are successfully copied.
(None, None, None, None)
Layer weights are successfully copied.
(None,)
Layer weights are successfully copied.


In [14]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-1.2588e-14, -2.4193e-14, -2.0195e-14,  ..., -1.7424e-14,
           -1.8165e-14, -2.0065e-14],
          [-4.0820e-14, -6.4369e-14, -5.6829e-14,  ..., -5.9112e-14,
           -5.9453e-14, -4.3775e-14],
          [-3.8931e-14, -5.8424e-14, -6.1809e-14,  ..., -5.6119e-14,
           -5.4058e-14, -5.3529e-14],
          ...,
          [-4.3402e-14, -6.5496e-14, -6.1202e-14,  ..., -5.4887e-14,
           -5.3833e-14, -4.7419e-14],
          [-4.1414e-14, -6.1010e-14, -6.2996e-14,  ..., -5.9068e-14,
           -5.5303e-14, -5.9356e-14],
          [-3.1195e-14, -5.2059e-14, -5.6881e-14,  ..., -4.7215e-14,
           -5.4061e-14, -4.4563e-14]],

         [[-1.1739e-14, -1.9304e-14, -1.9236e-14,  ..., -1.7454e-14,
           -1.9047e-14, -1.7575e-14],
          [-4.0282e-14, -6.3855e-14, -6.0149e-14,  ..., -5.6298e-14,
           -5.3430e-14, -6.7751e-14],
          [-3.9354e-14, -6.1378e-14, -6.11

At this point, the created "model_encoder" model contains the weights of the encoder part of the pre-trained Autoencoder model. 

## Obtaining the KDE representation of the training set images

In [15]:
########################################################
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
# encoded_images = model_encoder(images)
encoded_images = []

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [16]:
print(Y)

tensor([[[-1.6987e+00]],

        [[-1.4163e+00]],

        [[-1.8429e+01]],

        [[ 6.9137e-01]],

        [[-4.2074e-01]],

        [[ 1.7731e+00]],

        [[ 4.0843e+00]],

        [[-4.6746e-01]],

        [[-3.4074e-01]],

        [[-6.1546e+00]],

        [[-2.3728e+00]],

        [[-1.4119e+00]],

        [[-1.2111e-01]],

        [[ 9.7042e-01]],

        [[ 1.0873e+00]],

        [[-7.4112e+00]],

        [[ 9.8610e-01]],

        [[ 2.5965e+01]],

        [[ 1.0388e+02]],

        [[-7.5306e+00]],

        [[ 2.7499e+00]],

        [[ 3.0442e+00]],

        [[-7.3546e+00]],

        [[ 2.4517e+00]],

        [[-6.3275e-01]],

        [[ 1.0190e+00]],

        [[-3.8810e+01]],

        [[-1.9467e+00]],

        [[ 3.0416e+00]],

        [[-4.2009e-02]],

        [[ 2.5266e-01]],

        [[ 1.3165e+00]],

        [[ 6.6638e-01]],

        [[ 1.0562e+00]],

        [[ 6.0075e-01]],

        [[-5.0643e+00]],

        [[-7.8840e+00]],

        [[ 3.6839e-01]],

        [[-7

Convert "encoded_images" to a np array

In [17]:
np_encoded_images = []
for i in range (len(encoded_images)):
    # np_conversion = encoded_images[i].detach().numpy()        # If not using GPU
    np_conversion = encoded_images[i].cpu().detach().numpy()    # If using GPU
    np_encoded_images.append(np_conversion)
np_encoded_images = np.array(np_encoded_images)
print(type(np_encoded_images))

<class 'numpy.ndarray'>


In [18]:
print(np_encoded_images.shape)

(179, 450, 1, 1)


Also, see above the shape of the representation of the original images has been lowered to (1, 1) as specified in the model structure. The number 450, on the other hand, corresponds to the channels of the image; this value started at 3 and layer by layer it incremented until reaching 450.

Now, we have to flatten the data in order to apply kernel density on it.

In [19]:
model_encoder_output_shape = (450,1,1)
print(model_encoder_output_shape)

(450, 1, 1)


In [20]:
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]
encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_images]


In [21]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

The above function fits a kernel density estimation to the data that is provided, that is, to the "encoded_images_vector" variable. It does so using a Guassian kernel of bandwidth 0.2.

The badnwidth parameter affects on how the selected kernel will fit each sample of the given data. For example for the case in which the kernel is a Gaussian distribution, the bandwidth parameter would affect in how thin or wide is the Gaussian distribution.

At this point in the variable "kde" we have some numbers that are the result of fitting Gaussian functions to the given data points in the variable "encoded_images_vecotr". We will use the "kde" variable later for scoring with it, some given data points; the scoring will be given depending on how similar are the given data points to the ones that it had estimated.

Here below, it is shown the kde values corresponding to each encoded sample:

In [22]:
density_vals = kde.score_samples(encoded_images_vector)
print(density_vals)

[305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048203 306.23048203 306.23048204 306.23048204 306.23048203
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048

Notice that the above density values are pretty much the same among them. This has to do with the new model that was trained. The previous model from the "Pt_latentSpace_DS1" program, makes these value to be more different among them.

From here, the mean and standard deviation of these values are computed

In [23]:
average_density = np.mean(density_vals)
stdev_density = np.std(density_vals)
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)

The avg of the density values is:  305.69224305651085
The stdev_density of the density values is:  0.2887339073192086


Now, it will be shown the density mean and std deviation of the set of anomalies samples

In [24]:
data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the home laptop
# data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the work laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset_anomalies = datasets.ImageFolder(data_anomalies, transform=transform_characteristics)
dataloader_anomalies = torch.utils.data.DataLoader(dataset_anomalies, batch_size=32, shuffle=True)

In [25]:
#Get encoded output of input images = Latent space
encoded_anomalies_images = []

for i in range(len(dataset_anomalies)):
    X = dataset_anomalies[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_anomalies_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [26]:
np_encoded_anomaly_images = []
for i in range (len(encoded_anomalies_images)):
    # np_conversion = encoded_anomalies_images[i].detach().numpy()      # If GPU is not used
    np_conversion = encoded_anomalies_images[i].cpu().detach().numpy()    # If GPU is used
    np_encoded_anomaly_images.append(np_conversion)
np_encoded_anomaly_images = np.array(np_encoded_anomaly_images)
print(type(np_encoded_anomaly_images))

<class 'numpy.ndarray'>


In [27]:
encoded_anomaly_images_vector = [np.reshape(img_encoded, (out_vector_shape)) for img_encoded in np_encoded_anomaly_images]


In [28]:
print(encoded_anomaly_images_vector)

[array([-1.60451621e-01,  8.93159449e-01,  8.94589233e+00,  4.11879644e-03,
       -2.87828177e-01,  2.38115644e+00, -2.47116518e+00, -3.31508309e-01,
       -4.42461818e-01, -1.05557108e+00, -1.26283336e+00, -1.09436834e+00,
        6.37278676e-01, -9.31374550e-01,  4.52993751e-01, -1.58279121e+00,
        5.15620828e-01, -1.30081320e+01,  1.19035507e+02, -7.32630610e-01,
       -8.90363514e-01,  5.60010612e-01, -1.55057478e+00,  1.19282961e+00,
       -2.88298428e-01,  8.28526676e-01, -4.85010414e+01, -2.21356884e-01,
       -4.63841170e-01, -2.53187627e-01,  1.99598074e-01,  1.31547678e+00,
       -1.52599590e-03,  5.56890778e-02, -3.41079347e-02, -1.19551671e+00,
       -2.20600271e+00,  7.79398024e-01, -2.16844797e+00,  8.99743676e-01,
       -2.53381371e-01, -1.60368574e+00, -5.71211502e-02,  2.13625407e+00,
       -7.32899904e-01, -2.78955603e+00,  1.36376989e+00,  1.69476897e-01,
        7.26363182e-01,  9.86581564e-01, -6.86452985e-01, -7.45342016e-01,
        2.72901267e-01, 

In [29]:
density_vals_anomalies = kde.score_samples(encoded_anomaly_images_vector)
print(density_vals_anomalies)

[ 296.86804141   55.74787864 -628.07093602  182.55013798   74.70263388
  124.54903683]


In [30]:
average_density_anomalies = np.mean(density_vals_anomalies)
stdev_density_anomalies = np.std(density_vals_anomalies)
print("The avg of the density values is: ", average_density_anomalies)
print("The stdev_density of the density values is: ", stdev_density_anomalies)

The avg of the density values is:  17.724465453289927
The stdev_density of the density values is:  299.51185829842314


See that the std deviation along with the mean of these density values will overlap the mean of the non-anomaly images. 

Considering the situation of the overlapping commented above, given a density value, it will be built a function that:
- Assigns a percentage value according to the proximity to the mean of the non-anomaly images. For example: 
    - If the densitiy value is 305.69 (the mean of the density values of the non-anomaly images), then this density value should have 100% chance to be considered as non-anomaly.
    - If the densitiy value is 305.4014 (the mean of the density values of the non-anomaly minus the std deviation of the same set), then this density value should have 50% chance to be considered as non-anomaly.
- Assigns a percentage value according to the proximity to the mean of the anomaly images. For example: 
    - If the densitiy value is 231.6065 (the mean of the density values of the anomaly images), then this density value should have 100% chance to be considered as an anomaly image.
    - If the densitiy value is 311.0713 (the mean of the density values of the anomaly plus the std deviation of the same set), then this density value should have 50% chance to be considered as an anomaly.
- The two percentage values from above will be summed up assigning the following weights to the equation:

        = perc_NOanomaly*0.75 + perc_anomaly*0.25
        
    More weight is assigned to the non-anomaly images because there are more samples of this kind of images.
- After the weighted sum, the result will be subtracted from 100, to finally output the probability of an image to be an anomaly image. 

In [31]:
def ranges_mapper(value, leftMin, leftMax, rightMin, rightMax):
    # if(value>leftMax):
    #     return rightMax
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [32]:
def map_kde2prob_list(input_list):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images
    prob_score_list = []

    for i in range (len(input_list)):
        score_NOanomaly = input_list[i] - threshold_NOanomaly

        perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
        if perc_NOanomaly<0:
            perc_NOanomaly = 0
        if perc_NOanomaly>100:
            perc_NOanomaly = 100
        

        score_anomaly = input_list[i] - threshold_anomaly

        perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
        if perc_anomaly<0:
            perc_anomaly = 0
        if perc_anomaly>100:
            perc_anomaly = 100
        
        prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
        prob_score_list.append(prob_score)
    return prob_score_list

In [33]:
def map_kde2prob(value):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images          # The std deviation of the density values corresponding to the anomaly images

    score_NOanomaly = value - threshold_NOanomaly

    perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
    if perc_NOanomaly<0:
        perc_NOanomaly = 0
    if perc_NOanomaly>100:
        perc_NOanomaly = 100
    # print(perc_NOanomaly)

    score_anomaly = value - threshold_anomaly

    perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
    if perc_anomaly<0:
        perc_anomaly = 0
    if perc_anomaly>100:
        perc_anomaly = 100
    # print(perc_anomaly)
    prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
  
    return prob_score

In [34]:
def computePred(kde_value):
    pred = 0
    prob_anomaly = map_kde2prob(kde_value)/100
    if prob_anomaly > 0.5:
        pred = 1
    return pred

In [35]:
print("The prob of the kde value of being anomaly image is: ", map_kde2prob(average_density+stdev_density))
print("Given the probability, it is actually predicted as:", computePred(average_density+stdev_density))

The prob of the kde value of being anomaly image is:  49.530262889599314
Given the probability, it is actually predicted as: 0


Testing the kde thresholds 

In [36]:
test_dir = '../../../Images/BottleStoodUp_atNight/Evaluation'      #This is for the home laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])

dataset_test = datasets.ImageFolder(test_dir, transform=transform_characteristics)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=True)
classes = ('non-anomaly','anomaly')

In [37]:
#Get encoded output of input images = Latent space
encoded_test_imgs = []


for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()     
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    # np_conversion = Y.detach().numpy()
    np_conversion = Y.cpu().detach().numpy()
    encoded_test_imgs.append(np_conversion)
np_encoded_test_images = np.array(encoded_test_imgs)
print(type(np_encoded_test_images))

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


<class 'numpy.ndarray'>


In [38]:
encoded_test_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_test_images]
print(len(encoded_test_images_vector))

30


In [39]:
density_vals_test = kde.score_samples(encoded_test_images_vector)
print(density_vals_test)

[-5.11736839e+02 -1.31210584e+03 -3.39279602e+03 -3.35836069e+03
 -4.27298489e+03 -5.05722016e+03 -8.07041928e+03 -3.87431373e+03
 -1.66483204e+03 -3.17314176e+03  1.64738040e+01  2.99599957e+01
  2.19570014e+02 -3.15911504e+02 -8.94632259e+02 -1.04907844e+02
 -3.86342150e+04 -2.52793612e+04 -3.01238441e+03 -5.83318929e+02
 -6.32596110e+02 -8.41380167e+02 -1.00723879e+03 -9.48766222e+02
 -1.17822125e+03 -3.31851192e+03 -1.92413480e+03 -3.15036077e+03
 -3.56005743e+03 -2.17377019e+03]


The above shown values are REALLY strange. It was not expected to have negative values in the density numbers!

In [40]:
prob_test = map_kde2prob_list(density_vals_test)

In [41]:
print(prob_test)

[97.09684229914494, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 75.05219582286456, 75.51064465052976, 83.42393809079721, 88.92415524602495, 100.0, 80.11800725440213, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [42]:
y_pred = []
y_true = []

# Grabbing only the first image of the anomalies dataset
X = dataset_test[0]
image_in_tensor = X[0]
 
n_features = len(image_in_tensor[0])  # Get the size of one image of the anomaly images dataset. This is supposed to be 224
for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda() 
    ground_truth = X[1]

    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_converted_encoded_img = Y.cpu().detach().numpy()
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    prediction = computePred(density)
    y_pred.append(prediction) # Save Prediction
    y_true.append(ground_truth) # Save Truth
print(y_true)
print(y_pred)

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

Above, it can be seen that all the images are predicted to be anomaly images. This is good for the TPR but it is terrible for the FPR metric.